In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
data = pd.read_csv("train.csv")

label = []
for i in range(len(data)):
    if data.iloc[i]["LABEL"] == "SOFT_COVID":
        label.append(1)
    elif data.iloc[i]["LABEL"] == "STRONG_COVID":
        label.append(2)
    elif data.iloc[i]["LABEL"] == "DEAD":
        label.append(3)

data["LABEL"] = label

np.random.seed(42)
train = np.random.choice(len(data), int(len(data) * (1-0.2)), replace=False)
test = np.array([i for i in range(len(data)) if i not in train])
dataTest = data.loc[test]
data = data.loc[train]
dataTest = (dataTest.reset_index()).iloc[:,1:]
data = (data.reset_index()).iloc[:,1:]
data

,USMER,MEDICAL_UNIT,SEX,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,LABEL
0,2,4,1,2,12,2,2,2,2,2,2,2,2,2,2,2,2
1,1,12,2,2,66,2,2,2,2,2,1,2,2,2,2,2,1
2,2,4,1,2,55,2,2,2,2,2,1,2,2,2,2,2,1
3,2,12,1,2,53,2,1,2,2,2,2,2,2,2,2,2,1
4,2,12,1,2,47,2,2,2,2,2,1,2,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24035,2,8,2,1,46,2,2,2,2,2,2,2,2,2,2,2,3
24036,1,4,2,2,57,2,1,2,2,2,1,2,2,2,2,1,2
24037,2,12,2,1,75,2,2,2,2,2,2,2,2,2,2,2,2
24038,1,12,2,1,47,2,2,2,2,2,2,2,2,2,2,1,1


In [3]:
df = data.iloc[:,:16]
label = data["LABEL"]
df,label

(       USMER  MEDICAL_UNIT  SEX  PNEUMONIA  AGE  PREGNANT  DIABETES  COPD  \
 0          2             4    1          2   12         2         2     2   
 1          1            12    2          2   66         2         2     2   
 2          2             4    1          2   55         2         2     2   
 3          2            12    1          2   53         2         1     2   
 4          2            12    1          2   47         2         2     2   
 ...      ...           ...  ...        ...  ...       ...       ...   ...   
 24035      2             8    2          1   46         2         2     2   
 24036      1             4    2          2   57         2         1     2   
 24037      2            12    2          1   75         2         2     2   
 24038      1            12    2          1   47         2         2     2   
 24039      2             4    1          2   58         2         2     2   
 
        ASTHMA  INMSUPR  HIPERTENSION  OTHER_DISEASE  CARDIOVA

In [4]:
df_test = dataTest.iloc[:,:16]
label_test = dataTest["LABEL"]
df_test, label_test

(      USMER  MEDICAL_UNIT  SEX  PNEUMONIA  AGE  PREGNANT  DIABETES  COPD  \
 0         2             6    2          1   66         2         2     2   
 1         1             4    2          1   57         2         2     2   
 2         1            12    2          1   45         2         2     2   
 3         2            12    2          1   56         2         1     2   
 4         2             9    2          2   61         2         2     2   
 ...     ...           ...  ...        ...  ...       ...       ...   ...   
 6006      1             4    1          1   54         2         1     2   
 6007      2             4    1          1   79         2         2     2   
 6008      2             6    2          1   68         2         1     2   
 6009      1             4    2          2   76         2         1     2   
 6010      1             4    1          2   60         2         1     2   
 
       ASTHMA  INMSUPR  HIPERTENSION  OTHER_DISEASE  CARDIOVASCULAR  OBESI

In [5]:
def frequency(data, column):
    return {i:len(data[data[column] == i])/len(data) for i in np.unique(data[column])}

fLabel = frequency(data, "LABEL")
fParLab = {labels:{par:frequency(df[label == labels], par) for par in df.columns} for labels in fLabel}

In [6]:
def estimation(table):
    return (np.mean(table), np.var(table))

def loi_normale(x, mu, sigma2):
    return (1/((np.sqrt(sigma2) * np.sqrt(2*(np.pi)))) * np.exp(-((x - mu)**2)/(2 * sigma2)))

for labels in fLabel:
    mu, sigma2 = estimation(df[label == labels]["AGE"])
    for i in range(120):
        if i not in fParLab[labels]["AGE"]:
            fParLab[labels]["AGE"][i] = loi_normale(i, mu, sigma2)

fParLab[1]["MEDICAL_UNIT"][1] = 0
fParLab[1]["MEDICAL_UNIT"][2] = (fParLab[1]["MEDICAL_UNIT"][1] + fParLab[1]["MEDICAL_UNIT"][3])/2
fParLab[3]["MEDICAL_UNIT"][2] = (fParLab[3]["MEDICAL_UNIT"][1] + fParLab[3]["MEDICAL_UNIT"][3])/2
fParLab

{1: {'USMER': {1: 0.3375859912445278, 2: 0.6624140087554722},
  'MEDICAL_UNIT': {3: 0.022263914946841778,
   4: 0.300187617260788,
   5: 0.0061288305190744215,
   6: 0.038899312070043775,
   7: 0.0008755472170106316,
   8: 0.010631644777986242,
   9: 0.04040025015634772,
   10: 0.008630393996247656,
   11: 0.01125703564727955,
   12: 0.5599749843652283,
   13: 0.00075046904315197,
   1: 0,
   2: 0.011131957473420889},
  'SEX': {1: 0.4881801125703565, 2: 0.5118198874296436},
  'PNEUMONIA': {1: 0.12382739212007504, 2: 0.8761726078799249},
  'AGE': {0: 0.0012507817385866166,
   1: 0.000375234521575985,
   2: 0.00012507817385866166,
   3: 0.0013758599124452783,
   4: 0.0012507817385866166,
   5: 0.0012507817385866166,
   6: 0.0008755472170106316,
   7: 0.0005003126954346467,
   8: 0.0012507817385866166,
   9: 0.0012507817385866166,
   10: 0.001876172607879925,
   11: 0.0012507817385866166,
   12: 0.00150093808630394,
   13: 0.0013758599124452783,
   14: 0.0028767979987492183,
   15: 0.0030

In [7]:
def classifier(pars, fParLab, fLab):
    res = {}
    for label in fLab:
        res[label] = 1
        for p in fParLab[label]:
            res[label] *= fParLab[label][p][pars[p]] * fLab[label]
    return max(res, key=res.get)

def taux_bonne_classif(predictions, vrais_chiffres):
    return len(predictions[predictions == vrais_chiffres]) / len(predictions)

test = dataTest.iloc[:,:16]
testPred = np.array([classifier(test.iloc[i], fParLab, fLabel) for i in range(len(test))])
taux_bonne_classif(testPred, label_test)

0.6511395774413575

In [8]:
from sklearn.metrics import classification_report

print(classification_report(label_test, testPred))

              precision    recall  f1-score   support

           1       0.75      0.78      0.77      2022
           2       0.60      0.51      0.55      1988
           3       0.59      0.66      0.63      2001

    accuracy                           0.65      6011
   macro avg       0.65      0.65      0.65      6011
weighted avg       0.65      0.65      0.65      6011



In [9]:
def classifier_weighted(pars, fParLab, fLab, weight):
    res = {}
    for label in fLab:
        res[label] = 1
        for p in fParLab[label]:
            res[label] *= fParLab[label][p][pars[p]]**weight[p] * fLab[label]
    return max(res, key=res.get)

def findWeight(train, trainLabel, fParLab, fLabel, pas, n_iter=15):
    pars = list(fParLab.values())[0]
    dicti = {p:1 for p in fParLab[1]}
    wt = dict()
    wt = {a:dicti[a] for a in dicti}
    w = dict()
    w = {a:dicti[a] for a in dicti}
    for it in range(0,n_iter):
        taux_debut = taux_bonne_classif(np.array([classifier_weighted(train.iloc[i], fParLab, fLabel, w) for i in range(len(train))]), trainLabel)
        for p in pars:
            wt[p] = wt[p] + pas
            taux_wt = taux_bonne_classif(np.array([classifier_weighted(train.iloc[i], fParLab, fLabel, wt) for i in range(len(train))]), trainLabel)
            taux_w = taux_bonne_classif(np.array([classifier_weighted(train.iloc[i], fParLab, fLabel, w) for i in range(len(train))]), trainLabel)
            if taux_wt > taux_w:
                w[p] = w[p] + pas
            else:
                wt[p] = wt[p] - pas
        for p in pars:
            wt[p] = wt[p] - pas
            taux_wt = taux_bonne_classif(np.array([classifier_weighted(train.iloc[i], fParLab, fLabel, wt) for i in range(len(train))]), trainLabel)
            taux_w = taux_bonne_classif(np.array([classifier_weighted(train.iloc[i], fParLab, fLabel, w) for i in range(len(train))]), trainLabel)
            if taux_wt > taux_w:
                w[p] = w[p] - pas
            else:
                wt[p] = wt[p] + pas
        taux_fin = taux_bonne_classif(np.array([classifier_weighted(train.iloc[i], fParLab, fLabel, w) for i in range(len(train))]), trainLabel)
        print(str(it + 1) + "/" + str(n_iter) + " | Step used : " + str(pas) +" | Accuracy: " + str(taux_fin))
        if taux_fin == taux_debut:
            pas = pas/2
    return w

poids = findWeight(df, label, fParLab, fLabel, 0.4, 12)
poids

1/12 | Step used : 0.4 | Accuracy: 0.6699667221297837
2/12 | Step used : 0.4 | Accuracy: 0.6728369384359401
3/12 | Step used : 0.4 | Accuracy: 0.6731697171381031
4/12 | Step used : 0.4 | Accuracy: 0.6735440931780367
5/12 | Step used : 0.4 | Accuracy: 0.6735440931780367
6/12 | Step used : 0.2 | Accuracy: 0.6736688851913477
7/12 | Step used : 0.2 | Accuracy: 0.6740016638935108
8/12 | Step used : 0.2 | Accuracy: 0.6742096505823627
9/12 | Step used : 0.2 | Accuracy: 0.6742096505823627
10/12 | Step used : 0.1 | Accuracy: 0.6743344425956739
11/12 | Step used : 0.1 | Accuracy: 0.6743344425956739
12/12 | Step used : 0.05 | Accuracy: 0.6743344425956739


{'USMER': 1.4,
 'MEDICAL_UNIT': 1.4,
 'SEX': 1,
 'PNEUMONIA': 1.4,
 'AGE': 1.4,
 'PREGNANT': 0.7,
 'DIABETES': -0.20000000000000007,
 'COPD': -0.4000000000000001,
 'ASTHMA': 0.6,
 'INMSUPR': 1,
 'HIPERTENSION': 0.19999999999999996,
 'OTHER_DISEASE': 1.5999999999999999,
 'CARDIOVASCULAR': 0.19999999999999996,
 'OBESITY': 1,
 'RENAL_CHRONIC': 1,
 'TOBACCO': 1.4}

In [10]:
testPred = np.array([classifier_weighted(test.iloc[i], fParLab, fLabel, poids) for i in range(len(test))])
taux_bonne_classif(testPred, label_test)

0.6729329562468808

In [11]:
from sklearn.metrics import classification_report

print(classification_report(label_test, testPred))

              precision    recall  f1-score   support

           1       0.76      0.81      0.78      2022
           2       0.63      0.53      0.58      1988
           3       0.62      0.68      0.65      2001

    accuracy                           0.67      6011
   macro avg       0.67      0.67      0.67      6011
weighted avg       0.67      0.67      0.67      6011

